# File Operations

In addition to facilities for file copy (`%cp`), removal (`%rm`), etc, the IoT kernel supports copying entire libraries from the host to the microcontroller.

To make use of this feature, add a `projects` key to `devices.py` (customize the project folder names or download a sample setup from [github](https://github.com/iot49/iot49) and save the contents to `~/iot49`):

```python
device(uid = '30:ae:a4:28:39:f0', 
    name = 'my_esp',
    projects = [ 'airlift/mcu', 'boards/esp32/mcu'])
```

`projects` lists folder locations relative to `~/iot49` (or `$IOT49` if defined). The `%rsync` magic copies the contents of these folders to the microcontroller. The order matters: if a file with idential path exits in more than one source folder, only the last one is copied.

#### CircuitPython

By default, the flash on CircuitPython boards is accessible by USB and cannot be written to from Python. This prevents `%rsync` (and related magics) from working.

The simplest workaround is to add the following two lines to `boot.py`:

```python
import storage
storage.remount("/", False)
```

After that the flash can be written to by CircuitPython but is no longer accessible via the USB port (i.e. it won't mount as a drive on the host computer).

Alternatively, the flash drive can be manually ejected before issuing the `%rsync` magic. This does not require special code in `boot.py`.

#### MicroPython

MicroPython by default does not restrict access to the flash, although to avoid potential corruption it's important to ensure that flash drives are written to only by either MicroPython or the host.

On the pyboard this can be accomplished by adding

```python
import pyb
pyb.usb_mode('VCP')
```

to `boot.py`. This is optional - MicroPython is compatible with all features of the IoT Kernel "out of the box".

In [1]:
%connect my_esp
%rsync

Connected to my_esp @ serial:///dev/cu.usbserial-0163A3DA
COPY    lib/urpc/__init__.py
COPY    lib/ota32/__init__.py
COPY    lib/ota32/open_url.py
COPY    lib/urpc_demo.py
COPY    lib/urpc_server/__init__.py
COPY    lib/urpc/async_server.py
COPY    sample_main.py
COPY    lib/mp/__init__.py
COPY    lib/utelnetserver.py
COPY    lib/mp/server.py
COPY    lib/ota32/ota.py
COPY    lib/urpc_server/async_server.py
COPY    webrepl_cfg.py
COPY    boot.py


The `%rlist` magic shows all files currently stored on the microcontroller:

In [1]:
%rlist

   1824  Feb 10 18:38 2021  boot.py
                            lib/
                                mp/
     50  Feb 10 18:38 2021          __init__.py
   5227  Feb 10 18:38 2021          server.py
                                ota32/
     51  Feb 10 18:38 2021          __init__.py
   2048  Feb 10 18:38 2021          open_url.py
   3489  Feb 10 18:38 2021          ota.py
                                urpc/
     81  Feb 10 18:38 2021          __init__.py
   3163  Feb 10 18:38 2021          async_server.py
    805  Feb 10 18:38 2021      urpc_demo.py
                                urpc_server/
     81  Feb 10 18:38 2021          __init__.py
   3376  Feb 10 18:38 2021          async_server.py
   3884  Feb 10 18:38 2021      utelnetserver.py
    892  Feb 10 18:38 2021  sample_main.py
     52  Feb 10 18:38 2021  webrepl_cfg.py


If a file changes on the host (modification time and file size are used as criteria), re-run the `%rsync` command to upload the new version. The `%rdiff` command only shows the difference, but does not do any copying.

In [3]:
!touch $IOT49/boards/esp32/mcu/sample_main.py
%rdiff

UPDATE  sample_main.py


The `%rsync` magic only copies from the host to the microcontroller. By default, it deletes all files present on the microcontroller but not on the host. Specify the `-u` option to change this.

To copy files from the microcontroller to the host, use the `%cp` magic. `%cat` is another convenient magic that lists the contents of a file on the microcontroller:

In [3]:
%cat lib/mp/__init__.py

from .server import Server, start_server, VERSION



The `%rsync` command supports a few additional customization that are documented online. 